# SDSS Tutorial #4:  The Spatial Distribution of Galaxies

[Prof. Britt Lundgren, University of North Carolina Asheville](https://astrophysics.wp.unca.edu/britt-lundgren/)



## Objective
 - Using Sloan Digital Sky Survey (SDSS) data to explore how stars, galaxies, and the Milky Way's dusty interstellar medium are distributed in the sky.

## Learning Outcomes

By the end of this notebook, you should be able to: 
 - Use galactic coordinates to map the locations of celestial objects in the sky, relative to the Milky Way
 - Understand the reasons for differences in the spatial distributions of stars and galaxies on the sky
 - Understand how the interstellar medium of the Milky Way affects the number, brightness, and color of distant galaxies observed by the SDSS.
 - Understand the wavelength-dependence of dust extinction
 

## Prerequisites

* This notebook assumes that the user has basic proficiency in the **Python** programming language.
* This notebook uses **SQL queries** to select data from the SDSS database. If the SQL language and/or the SDSS database is new to you, we recommend this [brief tutorial](http://cas.sdss.org/dr16/en/help/howto/search/searchhowtohome.aspx). 
* This notebook is intended to be run in the [SciServer Compute](http://www.sciserver.org/tools/compute/) platform. If you have not used SciServer before, please review the [first notebook](https://escip.io/notebooks/sdss_tutorial_1.html) in this series of activities. 

Before we begin, let's fetch some required files:

In [1]:
# Import required images for this notebook

import os

if not os.path.exists('img'):
    !mkdir img
    
imgs = ['Realm_0079.jpg','galcoords.jpeg', 'gallat.jpeg', 'dr12_legacy_plus_segue_coverage.png', 'Realm_0084.jpg']
for i in imgs:
    if not os.path.exists('img/'+i):
        os.system('wget --no-check-certificate --content-disposition https://github.com/brittlundgren/ASTR420/raw/update_2023/img/'+i)
        os.system('mv %s img/' % i)
        
print('images imported')

images imported


**NOTE: In order for embedded images to display properly in this notebook, it may be necessary to restart the kernel and reopen this notebook before continuing. Do not re-compile the cell above after restarting.**


## Introduction

In [Chapter 3 of "The Realm of the Nebulae"](https://archive.org/details/in.ernet.dli.2015.210893/page/n75/mode/2up), Edwin Hubble discusses the large-scale distribution of "nebulae" on the sky.  

Below is a figure of Hubble's observations, which showed that at high galactic latitudes, the density of galaxies on the sky is appoximately constant, while numbers are suppressed at latitudes close to the Milky Way's disk, where their observation is obscured by high densities of stars and the interstellar medium.

<center><img src="img/Realm_0079.jpg"/></center>


The **galactic coordinate system** defines the Sun as the origin. Galactic longitude (l) is measured with primary direction from the Sun to the center of the galaxy in the galactic plane, while the galactic latitude (b) measures the angle of the object above the galactic plane.



<center><img src="img/gallat.jpeg"/></center>

In this tutorial, we will explore the distribution of stars and galaxies in the SDSS dataset, and also map the levels of dust extinction affecting distant galaxies, as a function of galactic latitude.
 
 Let's start, as always, by importing all of the most important libraries that we're likely to need...

In [2]:
# Import Python libraries to work with SciServer (see http://www.sciserver.org/docs/sciscript-python/)

import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive           # read/write to/from SciDrive, cloud-based storage for your work
import numpy as np                  # standard Python library for math operations
import pandas                       # package for structuring and visualizing data
import matplotlib.pyplot as plt     # package for graphing and visualization 
print('SciServer libraries imported')

# Apply some special settings to the imported libraries

# ensures columns get written completely in notebook
pandas.set_option('display.max_colwidth', -1)
# do *not* show python warnings 
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Image, SVG, Math
print('Settings applied')

# So that we're able to make plots, import matplotlib and some of its modules

import matplotlib
%config InlineBackend.rc = {}
from matplotlib import pylab as plt
%matplotlib inline
print('OK')


SciServer libraries imported
Settings applied
OK


-----------------

## Mapping celestial objects in galactic coordinates

Let's follow Hubble's lead and investigate how uniformly (or not) stars and galaxies appear to be in the sky.

The query below returns the number of photometric objects in a defined search area, in coordinates of RA and Dec:


In [36]:
# Here's an example query to count all of the objects in a rectangular search in RA and Dec coordinates.
#
query="""
SELECT count(*) as 'total',
    sum( case when (Type=3) then 1 else 0 end) as 'Galaxies',
    sum( case when (Type=6) then 1 else 0 end) as 'Stars',
    sum( case when (Type not in (3,6)) then 1 else 0 end) as 'Other'
FROM PhotoPrimary 
WHERE (ra between 265.4 and 267.4) and (dec between 30 and 31) 
"""
obj_counts = CasJobs.executeQuery(query, "dr16")
obj_counts

# Note: you can return the number of galaxies in the search area with the following command:
#
# print(obj_counts['Galaxies'][0])
#
# And you can return the total number of objects in the search area with:
# 
# print(obj_counts['total'][0])

,total,Galaxies,Stars,Other
0,1621,488,1133,0


**YOUR TURN: Modify the example query provided above to search for specified ranges in galactic coordinates (rather than in the equatorial coordinates of RA and Dec).** 

To find the column names for galactic latitude and longitude, refer to the [PhotoPrimary](https://skyserver.sdss.org/dr16/en/help/browser/browser.aspx#&&history=description+PhotoPrimary+V) table.

In [37]:
# YOUR CODE HERE

Note: that some choices of galactic coordinates will return no objects. A count of zero objects likely indicates that you have selected coordinates that are outside of the SDSS imaging footprint. Shown below is the SDSS Legacy and SEGUE Sky Coverage in equatorial coordinates (Credit: [SDSS](https://www.sdss4.org/dr16/))
<center><img src="img/dr12_legacy_plus_segue_coverage.png"/></center>



**Now, measure and plot the relative number density (e.g., the fraction of objects that are galaxies in a given area on the sky), as a function of galactic latitude.**  You might approach this by sampling areas of the sky at different galactic latitudes, each time measuring what fraction of the total number of objects are galaxies. Then put all of your measurements together in a plot so you can visualize and interpret the results.

Things to note:
- Due to the spherical nature of sky coordinates, measuring the area on the sky defined by a range of "b" and "l" (galactic latitude and longitude) is a little complicated.  By measuring the **relative number** of each type of object, you can get around needing to actually calculate the solid angle of sky that you're sampling when you make these measurements. 

- You should not restrict your search to objects with spectra; so you only need to query the [PhotoPrimary](https://skyserver.sdss.org/dr16/en/help/browser/browser.aspx#&&history=description+PhotoPrimary+V) table. 

In [5]:
# YOUR CODE HERE

**Does the fractional number of galaxies, you have plotted versus galactic latitude look similar to [Hubble's plot](https://archive.org/details/in.ernet.dli.2015.210893/page/n83/mode/2up) from "The Realm of the Nebulae" (shown below)?** 

Note that Hubble's plot shows the fraction of objects that are galaxies on the y-axis in **log-scale**. If you haven't already done that, [add a logarithmic scaling to your y-axis](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.semilogy.html).

<center><img src="img/Realm_0084.jpg"/></center>

**Discuss your findings in the cell below:**

Your interpretation here.

-------------
## Extinction and Reddening from Dust in the Interstellar Medium

Dust in the interstellar medium of the Milky Way serves to dim the light from background sources. We might expect that, if the universe is isotropic and homogeneous, then distant galaxies will have a similar mix of characteristics, regardless of their location on the sky.  Let's use this simple assumption to test the effect of dust reddening and extinction by the Milky Way.

**In the cell below, plot the mean g-r color of galaxies as a function of galactic latitude.**  (Make sure you are using the model magnitudes, without any extinction correction!)




In [41]:
# YOUR CODE HERE

**Interpret your findings in the cell below.**

Your interpretation here.

**In the cell below, plot the mean g magnitude of galaxies as a function of galactic latitude.**  (Make sure you are using the model magnitudes, without any extinction correction!)


In [ ]:
# YOUR CODE HERE

**Interpret your findings in the cell below.**

Your interpretation here.

**If you use the "de-reddened" magnitudes to calculate galaxy color, do your results change?**  Explore this question and plot of your findings below.


In [24]:
# YOUR CODE HERE

**Interpret your findings below**

Your interpretation here.

____
## Exploring the Wavelength-Dependence of Dust Reddening

The SDSS [PhotoPrimary table](https://skyserver.sdss.org/dr12/en/help/browser/browser.aspx#&&history=description+PhotoPrimary+V) provides measurements of dust extinction (e.g., "extinction_g") for each object in each of the 5 photometric filters: u,g,r,i, & z.  

**Measure and plot the mean extinction as a function of galactic latitude, for each of the photometric filters.** Overplot each of the filter measurements on the same figure.  Is the amplitude of the dust extinction the same or different, in each wavlength band?

In [32]:
# YOUR CODE HERE

**Interpret your findings in the cell below.**

Your interpretation here.